In [21]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.io as sio
import cv2
import matplotlib.image as mpimg
from matplotlib import path
from scipy.spatial.transform import Rotation 
from mpl_toolkits.mplot3d import Axes3D

In [22]:
DATA_DIR =  "data/data.mat"

data = sio.loadmat(DATA_DIR)['data']

In [23]:
x1 = data['x1'][0][0]; x2 = data['x2'][0][0]
x3 = data['x3'][0][0]; K = data['K'][0][0]
C = data['C'][0][0]; R = data['R'][0][0]
img1 = data['img1'][0][0].astype(np.uint8)
img2 = data['img2'][0][0].astype(np.uint8)
img3 = data['img3'][0][0].astype(np.uint8)

**Estimate Fundamental Matrix**

In [28]:
def vectorize_8_point(x1, x2):
    
    # Assume x1 = (u1,v1) and x2 =(u2,v2)
    u1 = x1[0]; u2 = x2[0]
    v1 = x2[1]; v2 = x2[1]

    return [
        u1*u2, u1*v2, u1, v1*u2, v1*v2, v1, u2, v2, 1
    ]

def enforce_singularity(F):
    
    U,S,Vd = np.linalg(F)
    #Set the last elemet of the diagonal of S to  zero and re
    
    S[-1] = 0
    
    F = U@np.diag(S)@(Vd)

    return F

def eight_point_algorithm(matched_points1, matched_points2, M=None):
    '''
    Algorithm to compute Fundamental Matrix of Rank 2 via SVD for (x2)'F(x1) = 0 using exactly 8 correspondence points. The eight point
    algorithm is based on paper by <insert reference here>. Usually, we need an overdetermined set of equations
    
    Key Steps:
    1) Scale the data by diving each coordinate by M (maxium of the image's width and heing)
    2) Estimate Fundamental Matrix by SVD
    3) Enforce singularity condition of F
    4) Unscale the fundamental Matrix
    5) normalise the fundamental matrix
    
    :param matched_points1(np.array) : size(8x2) matching points in image1
    :param matched_points2(np.array) : size(8x2) matching points in image2
    :param M:scale factor to scale the matched points (taken here as max of width and height of the image)
    
    Returns: 
        Fundamental Matrix (np.array) : size(3x3) of rank2 such that
    '''
    
       
    if M is not None:
        matched_points1 = matched_points1/M
        matched_points2 = matched_points2/M
    
    A = []
    
    for x,y in zip(matched_points1,matched_points2):
        A.extend(vectorize_8_point(x,y))
        
    A = np.array(A)
    
    U,S,Vd = np.linalg(A)
    
    # extract from the last column of Vd.T
    F_ = Vd.T[:,-1].reshape(3,3)
    
    # there is a refine 'F' step here; Not sure about that. Need to study some theory here. 
    print("Rank of the estimated Fundamental Matrix is : {}".format(np.linalg.matrix_rank(F_)))
        
    # F must be singular (remember, it is rank 2, since it isimportant for it to have a left and right nullspace, i.e. the epipoles).
    print("Enforcing the Rank Constraint for the Fundamental Matrix ..")
    
    F = enforce_singularity(F_)
    print("Rank of the re-estimated Fundamental Matrix is : {}".format(np.linalg.matrix_rank(F)))
    
    #unscale F by multiplying by the scale factor
    T = np.diag([1/M,1/M,1])
    F = T.T.dot(F).dot(T)
    
    #normalise the matrix such that ||F|| = 1
    F = F/np.norm(F)
    
    return F.reshape(3,3)    

In [ ]:
def estimateFundamentalMatrix_RANSAC(matched_points1, matched_points2):
    """
    
    Implemented with the help of following implementation : https://github.com/AdityaNair111/RANSAC-based-scene-geometry/blob/dcc2a5df87b71bb8cf3bd26c3ae9cc66afec522f/code/code_geo.py#L157
    
    Find the best Fundamental Matrix using RANSAC on potentially matching points
    
    :param matched_points1(np.array) : size(Nx2) matching points in image1
    :param matched_points2(np.array) : size(Nx2) matching points in image2
    Each row is a correspondense matching points
    Returns:
        - best_F: (np.array) size (3,3) representing the best fundamental matrix
        - inlier_a: (np.array) size(M,2) representing the subset of corresponding points from image A that are inliers with respect to best_F
        - inliers_b: (np.array) size(M,2) representing the subset of corresponding points from image B that are inliers with respect to best_F
    """    